# DB Cleaning

In [49]:
# Setup
source = "your_cup_of_cake"
unique_cols = ["title", "datePublished"]
select_cols = [x for x in unique_cols]
select_cols.append("_id")

In [50]:
# Determine Dups
df = pd.DataFrame(list(recipes.find({"source":source})))
print(f"Recipes Found: {len(df)}")

df = df[select_cols].groupby(unique_cols, as_index=False)
df = df["_id"].apply(list).reset_index(name="ids")

Recipes Found: 874


In [51]:
# Remove the duplicates
for index in tqdm(df.index):
    if len(df.loc[index, 'ids']) > 1:
        delete = [x for x in df.loc[index, 'ids'][1:]]
        for did in delete:
            recipes.delete_one({"_id":did})

In [52]:
print(f"Recipes After de-dup: {len(list(recipes.find({'source':source})))}")

Recipes After de-dup: 567


In [53]:
# by Title Alone
unique_cols = ["title"]
select_cols = [x for x in unique_cols]
select_cols.append("_id")
# Determine Dups
df = pd.DataFrame(list(recipes.find({"source":source})))
print(f"Recipes Found: {len(df)}")

df = df[select_cols].groupby(unique_cols, as_index=False)
sum(df.count()["_id"])

Recipes Found: 567


567